In [7]:
import pandas as pd
import pickle
import re
#Se importa el archivo pickle
with open('scrap_info_alq.pkl', 'rb') as f:
    diccionario = pickle.load(f)


with open('scrap_info.pkl', 'rb') as f:
    diccionario2 = pickle.load(f)
#Se transforma el diccionario en un dataframe
df = pd.DataFrame(diccionario)
df2 = pd.DataFrame(diccionario2)
#Ahora se cambia las filas por las columnas
df = df.T
df2 = df2.T
df.head()
df2.head()

#Ahora unimos los dos dataframes
df_def = pd.concat([df,df2])

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re

# Descargar los recursos necesarios para NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    if text is None:
        return ''
    # Convertir texto a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-záéíóúüñ\s]', '', text)
    # Tokenización
    tokens = nltk.word_tokenize(text)
    # Eliminar stopwords
    stop_words = set(stopwords.words('spanish'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lematización
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

descripcion_filtrada = df_def['Descripcion'].apply(preprocess_text)

print(descripcion_filtrada)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pjcan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pjcan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pjcan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


https://www.indomio.es/anuncios/95227781/    alquila piso amueblado habitaciones carrer del...
https://www.indomio.es/anuncios/95146207/    piso estudiantes entrar partir ro febrero juni...
https://www.indomio.es/anuncios/94613177/    disponible alquiler mensual anual bajo dormito...
https://www.indomio.es/anuncios/94500757/    disponible alquiler mensual anual bajo dormito...
https://www.indomio.es/anuncios/95223135/    rincón acogedor necesito espacio gustará aquí ...
                                                                   ...                        
https://www.indomio.es/anuncios/95007509/    atico m valencia valencia solidinmuebles vende...
https://www.indomio.es/anuncios/94958481/    venden pisos obra nueva camino moncada trata r...
https://www.indomio.es/anuncios/95006805/    vende exclusiva viviendas obra nueva calle arq...
https://www.indomio.es/anuncios/95313313/    vende piso barrio tormoseste luminoso apartame...
https://www.indomio.es/anuncios/95319871/    piso 

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

tv = TfidfVectorizer()
svd = TruncatedSVD(n_components=100)
lsa = make_pipeline(tv, svd, Normalizer(copy=False))
lsa_topic_vectors = lsa.fit_transform(descripcion_filtrada)

lsa_topic_vectors

array([[ 0.14350749, -0.26700188,  0.77941523, ..., -0.04421725,
        -0.07755961, -0.13903607],
       [ 0.19256545, -0.19861554,  0.03367638, ...,  0.07141233,
        -0.09746877, -0.06994394],
       [ 0.28819963, -0.31006392, -0.08226098, ..., -0.15500553,
         0.06717851, -0.05769826],
       ...,
       [ 0.25429298, -0.23135996, -0.06156733, ...,  0.0996736 ,
         0.02771597, -0.04458945],
       [ 0.39919404, -0.38600831,  0.0031658 , ..., -0.02268021,
        -0.10241295, -0.02080852],
       [ 0.31189253, -0.33007744,  0.01426828, ..., -0.11520907,
         0.06773613, -0.13084886]])

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

# Nueva descripción
descripcion = "Quiero un piso de unos 100 m2 que este bien iluminado, cueste menos de 700 euros al mes, este en la zona de campanar y tenga por lo menos 3 habitaciones y 2 baños"

# Transformar la nueva descripción usando la misma pipeline LSA
vector_descripcion = lsa.transform([descripcion])

# Calcular la similitud coseno entre la nueva descripción y todas las descripciones existentes
similarities = cosine_similarity(vector_descripcion, lsa_topic_vectors)

# Obtener los índices de las descripciones más similares en orden descendente
most_similar_indices = similarities.argsort()[0][::-1]

# Opcionalmente, puedes mostrar las descripciones más similares y sus similitudes
for index in most_similar_indices[:5]:  # muestra las 10 descripciones más similares
    print(f"Descripción: {df_def['Descripcion'].iloc[index]}")
    print(f"Similitud: {similarities[0][index]}")
    print()


Descripción: DISPONIBLE A PARTIR DE JULIO 2024. ALQUILER PARA ESTUDIANTES UNIVERSITARIOS (CONTRATO DE TEMPORADA). Se alquila este precioso apartamento , semireformado, en un edificio tranquilo y en una zona ideal si te gusta tener todo a un paso. Se trata de una vivienda con un total de 4 habitaciones y dos cuartos de baños completos. el apartamento se distribuye en zona de salón comedor con acceso a balcón y aire acondicionado (frío /calor), 4 habitaciones con camas de 1,35, totalmente equipadas y dos cuartos de baños completos. Cocina independiente, con todo lo que puedas necesitar en tu día a día y con acceso directo a terraza en zona común, pero de uso privativo. Como ves el piso lo tiene todo, es bonito, esta cerca de transporte público, tanto metro como autobús en menos de 9 minutos a pie, 5 minutos andando de supermercado y el mercado municipal del Cabañal a 7 minutos y lo mejor de todo, a tan solo 16 minutos a pie de la playa. No se puede pedir más. Si estás buscando una vivien